In [1]:
import lammps
import random
from tqdm import trange
import pymc3 as pm
import arviz as az
import numpy as np
import pymatgen as pmg
import theano.tensor as tt
import glob

from poscar_to_lammps import poscar_to_lammps
from pymatgen.io.vasp import Poscar

### Cell below reads in a POSCAR and outputs a lammps input config

In [3]:
# Defining our data:
# 1. Load a POSCAR using pymatgen
poscar = Poscar.from_file('poscars/POSCAR')
# 2. Define which elements are core-shell species
core_shell = { 'Li': False , 'Ni': False, 'O': True}
# 3. Set charges: dictionary values are either a single number for non-core-shell species, 
# or a sub-dictionary containing { 'core': core_charge, 'shell': shell_charge }
charges = {'Li': +1.0,
           'Ni': +3.0,
           'O': {'core':  +0.960,
                 'shell': -2.960}}
# poscar_to_lammps.
# Species.core_shell_index = 0
poscar_to_lammps(poscar, core_shell, charges )

C:\Users\Computer\Desktop\To_Be_Copied\R

180   atoms
60   bonds

4   atom types
1   bond types

0.0 5.701284 xlo xhi
0.0 12.141087 ylo yhi
0.0 13.911115 zlo zhi

Masses

4   6.94100 # Li 
5  58.69340 # Ni 
6  14.39946 # O core
7   1.59994 # O shell

Atoms

  1    1    4    1.000  -1.392659   12.353521   13.900686
  2    2    4    1.000   4.315252   2.639561   13.902764
  3    3    4    1.000  -2.824431   4.828940   0.020843
  4    4    4    1.000   1.464351   7.472875   13.878906
  5    5    4    1.000  -0.001539   9.722883   0.015179
  6    6    4    1.000   2.889570   0.259856   13.890251
  7    7    4    1.000   1.537607   2.468917   0.026340
  8    8    4    1.000   0.059156   4.811773   0.012627
  9    9    4    1.000  -1.395318   7.487645   13.899293
 10   10    4    1.000  -2.859056   9.692800   0.001597
 11   11    4    1.000   1.455580   0.911039   4.648476
 12   12    4    1.000   0.012955   3.292164   4.623660
 13   13    4    1.000  -1.368312   5.688208   4.632634
 14   14

In [ ]:
elements = ['Li', 'Ni', 'Oc', 'O']
# elements = pmg.Element('Li'), pmg.Element('Ni'), pmg.Element('Ge'), pmg.Element('O')
li, ni, oc, o = elements

pairs = [(li,o), (ni,o), (o,o)]

charges = {li: 1, ni: 3, oc: 0.960, o: -2.960}

bpp_def = {'Li_O_A'   : 456.783,
           'Li_O_rho' : 0.212,
           'Li_O_C'   : 0.0,
           'Ni_O_A'   : 1540.922,
           'Ni_O_rho' : 0.323,
           'Ni_O_C'   : 0.425,
           'O_O_A'    : 22764.7000,
           'O_O_rho'  : 0.1490,
           'O_O_C'    : 20.4}

bpp = {'Li_O_A'   : 456.783,
       'Li_O_rho' : 0.212,
       'Li_O_C'   : 0.0,
       'Ni_O_A'   : 1540.922,
       'Ni_O_rho' : 0.323,
       'Ni_O_C'   : 0.425,
       'O_O_A'    : 22764.7000,
       'O_O_rho'  : 0.1490,
       'O_O_C'    : 20.4}


In [ ]:
def init():
    lmp = lammps.Lammps(units='metal', style = 'full', args=['-log', 'none', '-screen', 'none'])
    lmp.command('fix csinfo all property/atom i_CSID')
    lmp.command('read_data poscars/coords.lmp fix csinfo NULL CS-Info')

    lmp.command('group cores type 3')
    lmp.command('group shells type 4')

    lmp.command('pair_style buck/coul/long/cs 12.0') #ADD THE CS IF USING CORESHELLS NOT IF RIGID ION
    lmp.command('pair_coeff * * 0 1 0')
    
    lmp.command('bond_style harmonic')
    lmp.command('bond_coeff 1 65.0 0.0')
    
    lmp.command('kspace_style pppm 1e-6')
    
    for c in charges:
        lmp.command('set atom %d charge %f' % (elements.index(c)+1, charges[c]))
        
    #setup for minimization
    lmp.command('min_style cg')

    return lmp

In [ ]:
def update_potentials(**kwargs):
    bpp.update(kwargs)
    
def set_potentials(instance):
    for pair in pairs:
        instance.command('pair_coeff {} {} {} {} {}'.format(dex(pair[0]),
                                                            dex(pair[1]),
                                                            bpp['{}_{}_A'.format(pair[0], pair[1])],
                                                            bpp['{}_{}_rho'.format(pair[0], pair[1])],
                                                            bpp['{}_{}_C'.format(pair[0], pair[1])]))

def dex(elin): return elements.index(elin) + 1

In [ ]:
def simfunc(**kwargs):
    if min(kwargs.values()) > 0:
        update_potentials(**kwargs)
        out = np.zeros([instances.system.total,3])
        
        set_potentials(instances)

        instances.command('fix 1 cores setforce 0.0 0.0 0.0')
        instances.command('minimize 1e-5 1e-5 5000 10000')
        instances.command('unfix 1')
        instances.run(0)
        
        out = instances.system.forces
            
    else: out = np.ones([instances.system.total,3])*999999999 # ThisAlgorithmBecomingSkynetCost
        
    return out

In [ ]:
instances = init()

In [ ]:
expected = np.zeros([instances.system.total,3])

In [ ]:
with pm.Model() as model:
#  pm.TruncatedNormal -- truncated so never tries a negative    pm.Uniform
    Li_O_A      = pm.Normal("Li_O_A",    mu = bpp_def['Li_O_A'],    sd =  100)
    Li_O_rho    = pm.Normal("Li_O_rho",  mu = bpp_def['Li_O_rho'],  sd =  0.05)
#     Li_O_C      = pm.Normal("Li_O_C",    mu = bpp_def['Li_O_C'],    sd =  0.01)
    Ni_O_A      = pm.Normal("Ni_O_A",    mu = bpp_def['Ni_O_A'],    sd =  100)
    Ni_O_rho    = pm.Normal("Ni_O_rho",  mu = bpp_def['Ni_O_rho'],  sd =  0.05)
    Ni_O_C      = pm.Normal("Ni_O_C",    mu = bpp_def['Ni_O_C'],    sd =  0.05)
    O_O_A       = pm.Normal("O_O_A",     mu = bpp_def['O_O_A'],     sd =  100)
    O_O_rho     = pm.Normal("O_O_rho",   mu = bpp_def['O_O_rho'],   sd =  0.05)
    O_O_C       = pm.Normal("O_O_C",     mu = bpp_def['O_O_C'],     sd =  5)
    
    simulator = pm.Simulator('simulator', simfunc, observed=expected)
    
    trace = pm.sample(step=pm.SMC(ABC=True, epsilon=0.1), draws=1000)


In [ ]:
az.style.use('arviz-darkgrid')
az.plot_trace(trace)

In [ ]:
az.plot_posterior(trace, round_to = 3, point_estimate = 'mode')